# Export agent movement (and create trips)

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
def get_locations(locations, times, show_trip = False): 
    new_locations = []
    last_loc = [-1,-1]
    last_time = -1

    for l,t in zip(locations, times):
        is_close = np.isclose(l, last_loc, rtol=1e-05, atol=1e-08, equal_nan=False)
        #print(is_close)
        if(last_time == -1):
            new_locations.append(l)
        elif (np.isnan(l[0])):
            #print("is nan")
            continue
        elif last_time != -1 and not (is_close[0] and is_close[1]) and  not (t == last_time): # time changed and place changed
            t_diff = int(t - last_time)
            x_diff = l[0] - last_loc[0] #- l[0]
            y_diff = l[1] - last_loc[1] #- l[1]
            i_x = (x_diff/np.float64(t_diff))
            i_y = (y_diff/np.float64(t_diff))
            # for each second of difference
            new_locations.extend([ [last_loc[0] + t*i_x, last_loc[1]+ t*i_y] for t in range(1,t_diff)])

        elif (is_close[0] and is_close[1]) and  not (t == last_time): # time changed but not place
            new_locations.extend([ [l[0],l[1]] for t in range(int(t - last_time))  ])

        last_loc = l
        last_time = t

    if(show_trip):
        G = nx.DiGraph() 
        last_node = -1
        for loc in new_locations:
            G.add_node(str(loc), x=float(loc[0]), y=float(loc[1]))
            if last_node != -1:
                G.add_edge(str(last_node),str(loc))
            last_node = loc
    
        positions = {}
        for idx,node in G.nodes(data=True):
            positions[idx] = [node['x'],node['y']]
    
        nx.draw_networkx_nodes(G, positions, node_color='r', alpha = 0.1, node_size = 10)
        fig_size=[5,5]
        plt.rcParams["figure.figsize"] = fig_size
        plt.axis('equal')
        plt.show()
    return new_locations


def prepare_agent(agent, show_trips = False):
    output = {}
    output['id'] = int(agent['id'])

    output['home'] = agent['home']
    output['home_time_from'] = int(agent['home_from'])
    output['home_time_to'] = int(agent['home_to'])

    output['work'] = agent['work']
    output['work_time_from'] = int(agent['work_from'])
    output['work_time_to'] = int(agent['work_to'])

    trips = []
    #print("\tNumber of trips:",len(agent["trips"]))
    for trip in agent["trips"]:
        new_trip = {}
        new_trip['from'] = trip["from"]
        new_trip['to'] = trip['to']
        new_trip['start'] = int(trip['start'])
        new_trip['vehicle'] = trip['vehicle']
        
        #interpolate locations
        new_trip['locations'] = get_locations(trip['locations'],trip['times'], show_trips)
        trips.append(new_trip)

    output['trips'] = trips
    return output

def save_output(ids, path, verbal = False):
    agents = []
    for id in ids:
        #try:
            agent = extract_agent(df.iloc[np.where(df.person_id == id)],False)
            if(len(agent)>0):
                out = prepare_agent(agent, verbal)
                agents.append(out)
                if(verbal):
                    for t in agent['trips']:
                        print("Start:",t["start"])
                        print("\t vehicle:", t["vehicle"])
                        #print("\t mode:", t["mode"])
                        print("\t from:", t["from"])
                        print("\t to:", t["to"])
                        print("\t length:", len(t["locations"]))
                    for t in out['trips']:
                    #    print("Start:",t["start"])
                    #    print("\t vehicle:", t["vehicle"])
                    #    print("\t mode:", t["mode"])
                        print("\t length (s):", len(t["locations"]))
        #except (RuntimeError, TypeError, NameError, IndexError):
        #    print("Error at:", id)
        #    return
        

    with open(path, 'w') as f:
        json.dump(agents, f)
        f.close()

    print("Agents saved to:",path)
    return
    

def get_facility_coords(df, facility_type="work"):
    #facility_types = ['work','home']
    home_coords = [-1,-1]
    time_from = 0.0; time_to = 0.0
    facility_events = df.iloc[np.where(df['actType'] == facility_type)]
    home_coords = [-facility_events.coords_x.unique()[0],-facility_events.coords_y.unique()[0]] 

    if(facility_type in ["home","work"] and facility_events.shape[0] >= 2):
        time_from = facility_events.iloc[np.where(facility_events.type == "actstart")].time.unique()[0]
        time_to = facility_events.iloc[np.where(facility_events.type == "actend")].time.unique()[0] 
        return home_coords, time_from, time_to 
    else:
        #print("Person did not arrive safely.")
        return home_coords, -1, -1

def return_coords(coords_x, coords_y, i_x, i_y):
    new_coords = [ [-float(x), - float(y)]
                            if not np.isnan(x)
                            else [-float(a), -float(b)] 
                            for x,y,a,b in zip(coords_x, coords_y, i_x, i_y)]
    return new_coords



In [3]:
def extract_agent(person, verbal=False):
    tmp = pd.DataFrame(person.copy())
    tmp.reset_index()
    person = tmp

    if("stuckAndAbort" in person.type.unique()):
        print("Agent",person.person_id.unique()[0], "is stuck.")
        return {}
       
    if("coords_to" in person.columns):
        person = person.drop(columns=["coords_from", "coords_to"])

    person["coords_from"] = return_coords(person.coords_x,person.coords_y, person.x_from, person.y_from)                 
    person["coords_to"] = return_coords(person.coords_x, person.coords_y, person.x_to, person.y_to)
    
    if(verbal):
        G = nx.DiGraph() 
        colors = ['g','k','b','orange','violet','lime','yellow','slateblue','tomato','indigo','olive']
        act_types = ["home","work","pt interaction", np.nan]
        special = ["home","work"]
        act_sizes = [300,300,100,5]
        act_colors = ["g","r","orange","b"]
        vehicles = []
        acts = set()

    agent = {}
    agent["id"] = person.person_id.unique()[0]
    h_coords, h_time_from, h_time_to = get_facility_coords(person, "home")
    if( h_time_from < 0):
        print("Agent",person.person_id.unique()[0], "did not arrive home.")
        return {}
    agent["home"] = h_coords
    agent["home_from"] = h_time_from
    agent["home_to"] = h_time_to

    w_coords, w_time_from, w_time_to = get_facility_coords(person, "work")
    if( w_time_from < 0):
        print("Agent",person.person_id.unique()[0], "did not arrive at work.")
        return {}
    agent["work"] = w_coords
    agent["work_from"] = w_time_from
    agent["work_to"] = w_time_to

    trips = []

    locations = []
    times = []
    trip = {}
    in_trip = False
    in_mhd = False
    in_car = False
    last_valid_loc = [-1,-1]

    for e, row in person.iterrows():
        #print(act, A, B)
        A = row.coords_from
        B = row.coords_to
        v = row.vehicle_id
        act = row.actType
        time = row.time
        event_type = row.type
        #if(event_type == 'departure'):
        if not in_trip and ((event_type == 'PersonEntersVehicle' and v.isnumeric()) or (event_type == "waitingForPt")): # add waiting for pt times
            in_trip = True
            trip = {}
            if(np.isnan(A[0])):
                trip['from'] = last_valid_loc
            else:
                trip['from'] = A
            trip['start'] = time
            locations = []
            times = []

        if in_trip and (event_type == 'PersonEntersVehicle' and not v.isnumeric()):
            in_mhd = True
        elif in_trip and (event_type == 'PersonEntersVehicle' and v.isnumeric()):
            in_car = True

        #if(event_type == 'arrival'):
        if(event_type == 'PersonLeavesVehicle' and in_trip and in_car) or (event_type == 'PersonLeavesVehicle' and in_trip and in_mhd):
            #print("vehicle:", v,"event type",event_type)
            if(np.isnan(B[0])):
                trip['to'] = last_valid_loc
            else:
                trip['to'] = B
            locations.append(B)
            times.append(time)
            trip['locations'] = locations
            trip['times'] = times
            if(v != 'nan' or not np.isnan(v)):
                if v.isnumeric():
                    trip['vehicle'] = "car"
                else:
                    trip['vehicle'] = v.split("_")[-1]
            else:
                trip['vehicle'] = None
            trips.append(trip)
            in_trip = False
            in_mhd = False
            in_car = False
            
        elif in_trip and not np.isnan(A[0]):
            times.append(time)
            locations.append(A)

        if(not np.isnan(B[0])):
            last_valid_loc = B

        if(verbal):
            acts.add(act)
            if act in special or (not str(A) in G and not np.isnan(A[0])):
                G.add_node(str(A), x=float(A[0]), y=float(A[1]), act_type=act_sizes[act_types.index(act)], act_color=act_colors[act_types.index(act)])
            if act in special or (not str(B) in G and not np.isnan(B[0])):
                G.add_node(str(B), x=float(B[0]), y=float(B[1]), act_type=act_sizes[act_types.index(act)], act_color=act_colors[act_types.index(act)])

            if (A[0] != B[0]) and (A[1] != B[1]) and not np.isnan(A[0]):
                if(v not in vehicles):
                    vehicles.append(v)
                #if(np.isnan(v) and legMode=="walk"): last_to -> walk_from
                G.add_edge(str(A),str(B), mode=colors[vehicles.index(v)])


    if(verbal):
        print(vehicles)
        #print(acts)
        positions = {}
        for idx,node in G.nodes(data=True):
            #print(idx, node)
            positions[idx] = [node['x'],node['y']]

        nx.draw_networkx_nodes(G, positions, node_color=[u['act_color'] for i,u in G.nodes(data=True)], alpha = 0.5, node_size = [u['act_type'] for i,u in G.nodes(data=True)])
        nx.draw_networkx_edges(G, positions, edge_color=[G[u][v]['mode'] for u,v in G.edges()],alpha=0.3, arrows = True)
        fig_size=[10,10]
        plt.rcParams["figure.figsize"] = fig_size
        plt.axis('equal')

    agent["trips"] = trips
    return agent

## Load and save one particular agent

In [4]:
df = pd.read_csv("./../output/plans/4.csv")
#df.info()
people = df.groupby("person_id")
p = 2078
display(df.iloc[np.where(df.person_id == p)].tail(15))
agent = extract_agent(df.iloc[np.where(df.person_id == p)].copy())

if(len(agent)>0):
    out = prepare_agent(agent, False)
    #print("agent prepped")
    save_output([p],'./../output/matsim_agents/agents_test.json', False)

,Unnamed: 0,time,type,link,vehicle_id,delay,actType,legMode,coords_x,coords_y,freespeed,oneway,x_from,y_from,x_to,y_to,person_id,facility,networkMode,relativePosition,transitLine,transitRoute,departure,atStop,destinationStop
13717,149,65466.0,entered link,41485,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,11.111111,1.0,749142.042336,1.041509e+06,749304.669223,1.041515e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13718,150,65481.0,entered link,41486,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,11.111111,1.0,749304.669223,1.041515e+06,749317.493638,1.041516e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13719,151,65483.0,entered link,41487,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,11.111111,1.0,749317.493638,1.041516e+06,749460.216192,1.041515e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13720,152,65496.0,entered link,483,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,11.111111,1.0,749460.216192,1.041515e+06,749484.145270,1.041506e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13721,153,65499.0,entered link,484,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,11.111111,1.0,749484.145270,1.041506e+06,749543.730920,1.041455e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13722,154,65506.0,VehicleArrivesAtFacility,NaN,veh_23415_tram,-134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2078,U85Z2P.link:484,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13723,155,65507.0,PersonLeavesVehicle,NaN,veh_23415_tram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13724,156,65507.0,arrival,484,NaN,NaN,NaN,pt,NaN,NaN,11.111111,1.0,749484.145270,1.041506e+06,749543.730920,1.041455e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13725,157,65507.0,actstart,484,NaN,NaN,pt interaction,NaN,749545.200599,1.041451e+06,11.111111,1.0,749484.145270,1.041506e+06,749543.730920,1.041455e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13726,158,65507.0,actend,484,NaN,NaN,pt interaction,NaN,749545.200599,1.041451e+06,11.111111,1.0,749484.145270,1.041506e+06,749543.730920,1.041455e+06,2078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Agents saved to: ./output/matsim_agents/agents_test.json


## Load and save all agents, chunk size depends on plans files
20 files - 10k population ~ 7 min

In [5]:
i = 0
last_ch = 0
for i,source in  enumerate(os.listdir('./../output/plans/')):
    if(i >= last_ch):
        #load source
        df = pd.read_csv("./../output/plans/"+str(i)+".csv")
        ids = df.person_id.unique()
        #print(ids)
        save_output(ids, './../output/matsim_agents/chunks500/agents_sec_'+str(i)+'.json')
        #break

Agents saved to: ./output/matsim_agents/chunks500/agents_sec_0.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_1.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_2.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_3.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_4.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_5.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_6.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_7.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_8.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_9.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_10.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_11.json
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_12.json
Agent 6788 did not arrive home.
Agents saved to: ./output/matsim_agents/chunks500/agents_sec_13.json
Agents saved